In [ ]:
%load_ext autoreload
%autoreload 2

# CNN Speckle filter

In [ ]:
from DatasetHandler import DatasetHandler
from CNNSpeckleFilter import CNNSpeckleFilter
from utils import *

## Load the dataset

In [ ]:
handler = DatasetHandler('dataset')
print("Training dataset size: ", len(handler.train_paths))
print("Validation dataset size: ", len(handler.val_paths))
print("testing dataset size: ", len(handler.test_paths))

In [ ]:
batch_speckle, batch_clean = next(iter(handler.data_loader(
    paths      = handler.train_paths, 
    batch_size = 1, 
    img_shape  = (256, 256))
))

In [ ]:
plot_dataset(batch_speckle, batch_clean)

## CNN speckle filter training

### Initialize the model

In [ ]:
speckle_filter = CNNSpeckleFilter(input_shape=(256,256,1), n_layers=12)
speckle_filter.model.summary()

### Train the model

In [ ]:
epochs = 100
batch_size = 16

train_gen = handler.data_loader(
    paths      = handler.train_paths, 
    batch_size = batch_size, 
    img_shape  = (256, 256))

val_gen = handler.data_loader(
    paths      = handler.val_paths, 
    batch_size = batch_size, 
    img_shape  = (256, 256))

train_step = len(handler.train_paths)//batch_size
val_step = len(handler.val_paths)//batch_size

history = speckle_filter.train_model(epochs, train_gen, val_gen, train_step, val_step)

### Plot training history

In [ ]:
plot_history(history)

## Test the model

### Plot results

In [ ]:
test_gen = handler.data_loader(
    paths      = handler.test_paths, 
    batch_size = 5, 
    img_shape  = (256, 256))

batch_speckle, batch_clean = next(iter(test_gen))
batch_pred = speckle_filter.model.predict(batch_speckle)

In [ ]:
plot_model_results(batch_speckle, batch_clean, batch_pred)

### Compute metrics

In [ ]:
compute_metrics(batch_speckle, batch_clean, batch_pred)